<table>
 <tr align=left><td><img align=left src="https://i.creativecommons.org/l/by/4.0/88x31.png">
 <td>Text provided under a Creative Commons Attribution license, CC-BY. All code is made available under the FSF-approved MIT license. (c) Marc Spiegelman, Template from Kyle Mandli</td>
</table>


# Fo-Fa Equilibrium Phase diagram

This notebook will use functions from the ThermoCodegen generated fo-fa system to construct an equilibrium binary phase loop

In [ ]:
# load the standard goodies
import numpy as np
from scipy.optimize import fsolve, brentq
import matplotlib.pyplot as plt
import matplotlib.cm as cm

%matplotlib inline
plt.rcParams["figure.figsize"] = [8,6]
plt.rcParams['font.size'] = 16

### Load the ThermoCodegen object for the fo-fa reactive system

This will require that the python bindings for the py_fo_fa_binary module are in the python path

if you are using modules,  this can be accomplished by running
|
```bash
cd ../../reactions/fo_fa_binary
module load ./fo_fa_binary.module


In [ ]:
import py_fo_fa_binary as py_fo_fa
py_fo_fa.phase_info()

In [ ]:
rxn = py_fo_fa.fo_fa_binary()
rxn.report()

In [ ]:
Ol_tcg = py_fo_fa.Olivine()
Lq_tcg = py_fo_fa.Liquid()

utility functions 


In [ ]:
toCelsius = lambda T: T - 273.15
toKelvin = lambda T: T + 273.15

### recalculate the equilibrium phase diagram

In [ ]:
def get_binary_loop(P,rxn,N):
    ''' 
    returns equilibrium liquidus and solid for ideal solution phases solid and liquid
    input:  
        P: pressure in bars
        rxn:  Reaction object from thermocodegen
        N:   Number of T points for evaluating 
    output:
        Trange:  numpy array of Temperatures
        x_l_eq:  Equilibrium liquidus concentrations (mol % Fo) at T,P
        x_s_eq:  Equilibrium solid concentrations at (mol % Fo) at T,P
    '''
    # calculate delta_mu of pure endmembers 
    # pure forsteritic liquid and solid
    CFo = rxn.zero_C()
    CFo[0] = [1., 0.]
    CFo[1] = [1., 0.]
    deltaMu0_0 = lambda T,P : rxn.A(T,P,CFo,0)
    CFa = rxn.zero_C()
    CFa[0] = [0., 1.]
    CFa[1] = [0., 1.]
    deltaMu0_1 = lambda T,P : rxn.A(T,P,CFa,1)

    # calculate melting points of pure endmembers at pressure p
    T_0 = fsolve(deltaMu0_0,1900.,(P))
    T_1 = fsolve(deltaMu0_1,1600.,(P))

    # sort
    T_min = np.min([T_0,T_1])
    T_max = np.max([T_0,T_1])
    print('T_min={} C, T_max={} C'.format(toCelsius(T_min),toCelsius(T_max)))
    
    #return T_min, T_max
    # set the temperature range for the two-phase region
    Trange = np.linspace(T_min+1.,T_max-1.,N)

    # will set full compositions just from c = [ cs_Fo, cf_Fo]
    C = lambda c : np.array([[c[0], 1.-c[0]], [c[1], 1.-c[1]]])
    # dummy phase values to make sure Gamma is never zero except when A=0
    Phi = [1., 1.]
    
    # Find zeros of Reaction terms
    As = lambda c,T,P : rxn.Gamma_ik(T,P,C(c),Phi)[0]


    # set initial guess for compositions as near pure fayalite
    cs = 0.001
    cl = 0.001
    c0 = np.array([cs,cl])

    c_eq = np.empty((len(Trange),2))
    tol = 1.e-8

    # loop over Temperature and calculate equilibrium liquid/solid concentrations in wt% at t,p
    for i,T in enumerate(Trange):
    
        # find zeros of As at (T,P) given initial guess c0 
        c_eq[i] = fsolve(As, c0, (T,P))
    
        # check that the solver converged
        error = np.linalg.norm(As(c_eq[i],T,P))
        if error > tol:
            print('Error: not converged at P={},T={} C: ||A(x,T)||={}'.format(P,toCelsius(T),np.linalg.norm(As(c_eq[i],T,P))))

        # set new guess to last solution
        c0 = c_eq[i]
    
    return Trange, c_eq.T

## Phase Diagram

calculate the binary phase diagram at fixed pressure


In [ ]:
P = 1000.
Trange,c_eq = get_binary_loop(P,rxn,1001)

### plot it out

In [ ]:
fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(1,1,1)
ax.set_title("Binary Phase Diagram: P={} bars".format(P), fontsize = 18)
ax.set_xlabel("$c_{fo}$", fontsize = 24)
ax.set_ylabel("$T  (^{\\circ}C)$", fontsize = 18)
ax.plot(c_eq[0,:], toCelsius(Trange), '-r', label = "solidus")
ax.plot(c_eq[1,:], toCelsius(Trange), '-b', label = "liquidus")
ax.set_xlim([0.0, 1.0])
ax.grid()
ax.legend(loc='best')
plt.show()

## Comparison with ThermoEngine Equilibrate for the same system

Note:  to run this notebook beyond this point will require a version of thermoengine that allows integration with ThermoCodegen

In [ ]:
from thermoengine import model, equilibrate, core

In [ ]:
import thermoengine

## Create phases for equilibrium assemblages


In [ ]:
modelDB = model.Database.from_thermocodegen(py_fo_fa)
modelDB.phase_info

In [ ]:
Liquid = modelDB.get_phase('Liq')
Ol = modelDB.get_phase('Ol')

## Define elements in system and phases in system


In [ ]:
# question is whether these need to be in periodic order
elm_sys = [ 'O', 'Mg', 'Si', 'Fe']
phs_sys = [Liquid,  Ol]

## Composition of the system
Need to calculate the bulk composition in moles of liquid endmembers I think

Liquid and Solid endmembers are Mg2SiO4 and Fe2SiO4


In [ ]:
T = toKelvin(1600.)
Cblk = [ 0.4, 0.6]
lq = py_fo_fa.Liquid()
mols = lq.c_to_x(Cblk)
formula = Liquid.compute_formula(T, P, mols)
print(mols, formula)

In [ ]:
mol_elm = Liquid.covert_endmember_comp(mols, output='moles_elements')
#mol_elm


Cast this composition as moles of elements for input to the Equilibrate class

In [ ]:
blk_cmp = []
for elm in elm_sys:
    index = core.chem.PERIODIC_ORDER.tolist().index(elm)
    blk_cmp.append(mol_elm[index])
blk_cmp = np.array(blk_cmp)
print(elm_sys)
print(blk_cmp)
#blk_cmp = np.array([4., 1., 1., 1.])

## Instantiate equilibrate class instance and run calculation

In [ ]:
equil = equilibrate.Equilibrate(elm_sys, phs_sys)

In [ ]:
state = equil.execute(T, P, bulk_comp=blk_cmp, debug=0, stats=True)
state.print_state()

In [ ]:
Lq_C_Fo = state.compositions(phase_name='Liquid',units='wt%')[0]/100.
Ol_C_Fo = state.compositions(phase_name='Olivine',units='wt%')[0]/100.
print(Lq_C_Fo, Ol_C_Fo)

In [ ]:
Lq_X_Fo = state.compositions(phase_name='Liquid',units='mole_frac')[0]
Ol_X_Fo = state.compositions(phase_name='Olivine',units='mole_frac')[0]
print(Lq_X_Fo, Ol_X_Fo)

and plot on the phase diagrams

### plot it out

In [ ]:
fig = plt.figure(figsize=(16,6))
ax = fig.add_subplot(1,2,1)
ax.set_title("Binary Phase Diagram: P={}".format(P), fontsize = 18)
ax.set_xlabel("$c_{fo}$", fontsize = 24)
ax.set_ylabel("$T  (^{\\circ}C)$", fontsize = 18)
ax.plot(c_eq[0,:], toCelsius(Trange), '-r', label = "solidus")
ax.plot(c_eq[1,:], toCelsius(Trange), '-b', label = "liquidus")
ax.plot(Ol_C_Fo, toCelsius(T),'ro',markersize=10,label='Equilibrate')
ax.plot(Lq_C_Fo, toCelsius(T),'bo',markersize=10,label='Equilibrate')

ax.set_xlim([0.0, 1.0])
ax.grid()
ax.legend(loc='best')

ax = fig.add_subplot(1,2,2)
ax.set_title("Binary Phase Diagram: P={}".format(P), fontsize = 18)
ax.set_xlabel("$x_{fo}$", fontsize = 24)
ax.set_ylabel("$T  (^{\\circ}C)$", fontsize = 18)
xs_eq = np.array([ Ol_tcg.c_to_x([c, 1-c])[0] for c in c_eq[0] ])
xl_eq = np.array([ Lq_tcg.c_to_x([c, 1-c])[0] for c in c_eq[1] ])
ax.plot(xs_eq, toCelsius(Trange), '-r', label = "solidus")
ax.plot(xl_eq, toCelsius(Trange), '-b', label = "liquidus")
ax.plot(Ol_X_Fo, toCelsius(T),'ro',markersize=10,label='Equilibrate')
ax.plot(Lq_X_Fo, toCelsius(T),'bo',markersize=10,label='Equilibrate')
ax.set_xlim([0.0, 1.0])
ax.grid()
ax.legend(loc='best')
plt.show()